In [17]:
## dependencies
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, utils
import matplotlib.pyplot as plt
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [18]:
## Constants
PLACE_PULSE_PATH ='votes_clean.csv'
IMAGES_PATH= 'placepulse/'
MODEL_PATH = 'model.pth'

In [27]:
## Data loader class

class PlacePulseDataset(Dataset):
    
    def __init__(self,csv_file,img_dir,transform=None, cat=None, equal=False):
        self.placepulse_data = pd.read_csv(csv_file)
        if cat:
            self.placepulse_data = self.placepulse_data[self.placepulse_data['category'] == cat]
        if not equal:
            self.placepulse_data = self.placepulse_data[self.placepulse_data['winner'] != 'equal']
        
        self.img_dir =  img_dir
        self.transform = transform
        self.label = {'left':1, 'right':-1,'equal':0}
    
    def __len__(self):
        return len(self.placepulse_data)
    
    def __getitem__(self,idx):
        
        if type(idx) == torch.Tensor:
            idx = idx.item()
        
        left_img_name = os.path.join(self.img_dir, f'{self.placepulse_data.iloc[idx, 0]}.jpg')
        left_image = io.imread(left_img_name)
        right_img_name = os.path.join(self.img_dir, f'{self.placepulse_data.iloc[idx, 1]}.jpg')
        right_image = io.imread(right_img_name)
        winner = self.label[self.placepulse_data.iloc[idx, 2]]
        cat = self.placepulse_data.iloc[idx, -1]
        sample = {'left_image': left_image, 'right_image':right_image,'winner': winner, 'cat':cat}
        if self.transform:
            sample = self.transform(sample)
        return sample

In [20]:
#  Transformers 

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        left_image, right_image = sample['left_image'], sample['right_image']
        
        return {'left_image': ToTensor.transform_image(left_image),
                'right_image': ToTensor.transform_image(right_image),
                'winner': sample['winner'],
                'cat': sample['cat']}
    @classmethod
    def transform_image(cls,image):
        return torch.from_numpy(image.transpose((2, 0, 1))).float()
    
class Rescale():
    
    def __init__ (self,output_size):
        self.output_size = output_size
    
    def __call__(self, sample):
        left_image, right_image = sample['left_image'], sample['right_image']
        
        return {'left_image': transform.resize(left_image,self.output_size,anti_aliasing=True,mode='constant'),
                'right_image': transform.resize(right_image,self.output_size,anti_aliasing=True,mode='constant'),
                'winner': sample['winner'],
                'cat': sample['cat']}
        


In [ ]:
# seeing images for testing
%matplotlib inline
import random

data = PlacePulseDataset(PLACE_PULSE_PATH,IMAGES_PATH,transforms.Compose([Rescale((224,224))]))
fig = plt.figure()

for i in range(4):
    sample = data[random.randint(0,len(data))]

    print(sample['left_image'].shape, sample['right_image'].shape)

    ax = plt.subplot(1, 4, i + 1)
    plt.tight_layout()    
    ax.set_title('Sample #{}'.format(i))
    plt.imshow(sample['left_image'])
    ax.axis('off')

    if i == 3:
        plt.show()
        break

In [6]:
#SsCnn definition

class SsCnn(nn.Module):
    
    def __init__(self):
        super(SsCnn, self).__init__()
        #shouldbe vgg19
        self.cnn = models.alexnet(pretrained=True).features
        #self.cnn.train() # to finetune pretrained model
        self.fuse_conv_1 = nn.Conv2d(512,512,3)
        self.fuse_conv_2 = nn.Conv2d(512,512,3)
        self.fuse_conv_3 = nn.Conv2d(512,512,3)
        self.fuse_fc = nn.Linear(512*4*8, 2)
        self.classifier = nn.LogSoftmax(dim=1)
        
        
    
    def forward(self,left_image, right_image):
        left = self.cnn(left_image)
        right = self.cnn(right_image)
        x = torch.cat((left,right),1)
        x = self.fuse_conv_1(x)
        x = self.fuse_conv_2(x)
        x = self.fuse_conv_3(x)
        x = x.view(4,512*4*8)
        x = self.fuse_fc(x)
        x = self.classifier(x)
        return x

In [7]:
# SsCnn Training test
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = SsCnn()
net = net.to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        # get the inputs
        input_left, input_right, label = data['left_image'], data['right_image'], data['winner']
        input_left, input_right, label = input_left.to(device), input_right.to(device), label.to(device)
        label[label==-1] = 0
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(input_left,input_right)

        loss = criterion(outputs, label)
        loss.to(device)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')


NameError: name 'dataloader' is not defined

In [21]:
#SsCnn definition

class RSsCnn(nn.Module):
    
    def __init__(self):
        super(RSsCnn, self).__init__()
        #shouldbe vgg19
        self.cnn = models.alexnet(pretrained=True).features
        #self.cnn.train() # to finetune pretrained model
        self.fuse_conv_1 = nn.Conv2d(512,512,3)
        self.fuse_conv_2 = nn.Conv2d(512,512,3)
        self.fuse_conv_3 = nn.Conv2d(512,512,2)
        self.fuse_fc = nn.Linear(512, 2)
        self.classifier = nn.LogSoftmax(dim=1)
        self.rank_fc_1 = nn.Linear(256*6*6, 4096)
        self.rank_fc_2 = nn.Linear(4096, 1)
    
    def forward(self,left_image, right_image):
        left = self.cnn(left_image)
        right = self.cnn(right_image)
#         print(left.size())
        x = torch.cat((left,right),1)
        x = self.fuse_conv_1(x)
        x = self.fuse_conv_2(x)
#         print(x.size())
        x = self.fuse_conv_3(x)
#         print(x.size())
        x = x.view(32,512)
        x_clf = self.fuse_fc(x)
        x_clf = self.classifier(x_clf)
        x_rank_left = left.view(32,256*6*6)
        x_rank_right = right.view(32,256*6*6)
        x_rank_left = self.rank_fc_1(x_rank_left)
        x_rank_right = self.rank_fc_1(x_rank_right)
        x_rank_left = self.rank_fc_2(x_rank_left)
        x_rank_right = self.rank_fc_2(x_rank_right)
        return x_clf,x_rank_left, x_rank_right

In [28]:
data=PlacePulseDataset(PLACE_PULSE_PATH,IMAGES_PATH,transforms.Compose([Rescale((224,224)),ToTensor()]),'wealthy')
len_data = len(data)
train_len = int(len_data*0.65)
val_len = int(len_data*0.05)
test_len = len_data-train_len-val_len
train,val,test = random_split(data,[train_len , val_len, test_len])
print(len(train))
print(len(val))
print(len(test))
dataloader = DataLoader(train, batch_size=32,
                        shuffle=True, num_workers=4)

85506
6577
39466


In [29]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = RSsCnn()
net = net.to(device)

In [30]:
#torch ignite resume training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

MODEL_PATH='models/test_model_state_dict_4.pth'
OPTIMIZER_PATH='models/test_optimizer_state_dict_4.pth'



net = RSsCnn()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
net.load_state_dict(torch.load(MODEL_PATH))
optimizer.load_state_dict(torch.load(OPTIMIZER_PATH))
epoch = 1


net.train()
net = net.to(device)

In [ ]:
# training with torch ignite
from ignite.engine import Engine, Events, create_supervised_evaluator
from ignite.metrics import Accuracy,Loss, RunningAverage
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import ModelCheckpoint

clf_crit = nn.NLLLoss()
rank_crit = nn.MarginRankingLoss(reduction='sum')
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.9)
lamb = 0.5

def update(engine, data):
    input_left, input_right, label = data['left_image'], data['right_image'], data['winner']
    input_left, input_right, label = input_left.to(device), input_right.to(device), label.to(device)
    rank_label = label.clone()
    label[label==-1] = 0
    # zero the parameter gradients
    optimizer.zero_grad()
    rank_label = rank_label.float()
    # forward + backward + optimize
    output_clf,output_rank_left, output_rank_right = net(input_left,input_right)

    loss_clf = clf_crit(output_clf,label)
#   print(output_rank_left, output_rank_right, rank_label)
    loss_rank = rank_crit(output_rank_left, output_rank_right, rank_label)
    loss = loss_clf + loss_rank*lamb
    loss.to(device)
    loss.backward()
    optimizer.step()
    return  { 'loss':loss.item(), 
             'loss_clf':loss_clf.item(), 
             'loss_rank':loss_rank.item(),
             'y':label,
             'y_pred': output_clf
            }

trainer = Engine(update)

RunningAverage(output_transform=lambda x: x['loss']).attach(trainer, 'loss')
RunningAverage(Accuracy(output_transform=lambda x: (x['y_pred'],x['y']))).attach(trainer,'avg_acc')
pbar = ProgressBar(persist=True)
pbar.attach(trainer,['loss','avg_acc'])

handler = ModelCheckpoint('models', 'test', save_interval=2, n_saved=2, create_dir=True, save_as_state_dict=True, require_empty=False)
trainer.add_event_handler(Events.EPOCH_COMPLETED, handler, {
            'model': net,
            'optimizer': optimizer,
            })
    
trainer.run(dataloader,max_epochs=1)

Epoch [1/1]: [56/2673]   2%|▏         , avg_acc=4.94e-01, loss=6.95e-01 [01:17<37:00]  